# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [16]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
import gmaps.datasets
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [17]:
df_path = "output_data/cities.csv"
cities_data = pd.read_csv(df_path)
cities_data

,Unnamed: 0,City,City Lat,City Long,Max Temp,Humidity,Cloud Coverage,Wind Speed,City Country,City Datetime
0,0,Bluff,-46.6000,168.3333,53.01,88,100,4.00,NZ,1618367277
1,1,Port Elizabeth,-33.9180,25.5701,66.20,94,90,10.36,ZA,1618367278
2,2,Rikitea,-23.1203,-134.9692,77.70,66,7,6.82,PF,1618367278
3,3,Zhigansk,66.7697,123.3711,19.54,87,100,21.00,RU,1618367279
4,4,Bandarbeyla,9.4942,50.8122,78.49,75,72,5.41,SO,1618367279
...,...,...,...,...,...,...,...,...,...,...
577,577,Hobyo,5.3505,48.5268,80.96,66,99,8.30,SO,1618367575
578,578,Clifton,40.8584,-74.1638,57.99,62,90,2.46,US,1618367298
579,579,Otjiwarongo,-20.4637,16.6477,60.31,41,0,15.37,NaN,1618367576
580,580,Béziers,43.3476,3.2190,39.20,70,0,6.91,FR,1618367576


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [18]:
#Access maps with my unique API key
gmaps.configure(api_key=g_key)

#Use Latitude and Longitude from weather dataframe as locations
locations = cities_data[["City Lat", "City Long"]]

#Use humidity from dataframe as the weight
humidity = cities_data["Humidity"]


In [19]:
#create heatmap layer
fig = gmaps.figure(center=(25,0), zoom_level = 1)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating=False, max_intensity=100, point_radius = 1)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#generate a new dataframe consiting of cities that have ideal weather by filtering cities_data by desirable weather attributes
ideal_destinations_df = cities_data.loc[(cities_data["Max Temp"] > 70) & 
                               (cities_data["Max Temp"] < 90) & 
                               (cities_data["Wind Speed"] < 15) & 
                               (cities_data["Cloud Coverage"] == 0) &
                               (cities_data["Humidity"] < 60), :].reset_index(drop=True).dropna()
ideal_destinations_df

,Unnamed: 0,City,City Lat,City Long,Max Temp,Humidity,Cloud Coverage,Wind Speed,City Country,City Datetime
0,157,Fatehpur,25.9333,80.8000,89.26,11,0,4.16,IN,1618367354
1,159,Byron Bay,-28.6500,153.6167,75.20,43,0,9.22,AU,1618367290
2,166,Kaffrine,14.1059,-15.5508,76.21,36,0,7.09,SN,1618367359
4,231,Maibong,25.3000,93.1667,84.27,31,0,4.52,IN,1618367394
5,239,Tura,25.5198,90.2201,82.94,40,0,7.07,IN,1618367397
7,380,Bilma,18.6853,12.9164,80.20,8,0,9.46,NE,1618367474
8,467,Gaya,24.7833,85.0000,80.60,24,0,5.82,IN,1618367314


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#Create a dataframe to store acceptable city hotels in 
hotel_df = ideal_destinations_df.loc[:,["City", "City Country", "City Lat", "City Long"]]

#add column with header name "Hotel"
hotel_df["Hotel"] = ""
hotel_df

,City,City Country,City Lat,City Long,Hotel
0,Fatehpur,IN,25.9333,80.8000,
1,Byron Bay,AU,-28.6500,153.6167,
2,Kaffrine,SN,14.1059,-15.5508,
4,Maibong,IN,25.3000,93.1667,
5,Tura,IN,25.5198,90.2201,
7,Bilma,NE,18.6853,12.9164,
8,Gaya,IN,24.7833,85.0000,


In [7]:
#
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}


for index, row in hotel_df.iterrows():
    # get city name, lat, long from hotel_df
    i_lat = row["City Lat"]
    i_lng = row["City Long"]
    city_name = row["City"]
    
    # add keyword to parameters dictionary
    params["location"] = f"{i_lat},{i_lng}"

    #make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # pull results
    results = response['results']
    
    # save the hotel name to a dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 0: Fatehpur.
Closest hotel in Fatehpur is Era Garden.
------------
Retrieving Results for Index 1: Byron Bay.
Closest hotel in Byron Bay is Atlantic Byron Bay.
------------
Retrieving Results for Index 2: Kaffrine.
Closest hotel in Kaffrine is Hotel de Kaffrine.
------------
Retrieving Results for Index 4: Maibong.
Missing field/result... skipping.
------------
Retrieving Results for Index 5: Tura.
Closest hotel in Tura is Hotel C&C.
------------
Retrieving Results for Index 7: Bilma.
Missing field/result... skipping.
------------
Retrieving Results for Index 8: Gaya.
Closest hotel in Gaya is Hotel Gharana Tree - Best Hotel in Gaya.
------------
-------End of Search-------


In [8]:
hotel_df

,City,City Country,City Lat,City Long,Hotel,Hotel Name
0,Fatehpur,IN,25.9333,80.8000,,Era Garden
1,Byron Bay,AU,-28.6500,153.6167,,Atlantic Byron Bay
2,Kaffrine,SN,14.1059,-15.5508,,Hotel de Kaffrine
4,Maibong,IN,25.3000,93.1667,,NaN
5,Tura,IN,25.5198,90.2201,,Hotel C&C
7,Bilma,NE,18.6853,12.9164,,NaN
8,Gaya,IN,24.7833,85.0000,,Hotel Gharana Tree - Best Hotel in Gaya


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{City Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["City Lat", "City Long"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig


Figure(layout=FigureLayout(height='420px'))